In [9]:
import pymongo
import pandas as pd
import random
import json
from os.path import join,dirname
import os
from pymongo import MongoClient
import random
from datetime import date, timedelta, datetime
DATEFORMAT = "%Y-%m-%d"

client = MongoClient('localhost', 27017)
db = client['aion']
collection = db['employee']

ROOT_DIR = os.path.abspath(os.curdir)
filename =join(ROOT_DIR,'../../data/employee_mock_data.json')
if filename:
    with open(filename, 'r') as f:
        employees = json.load(f)
print(employees[0:2])
# bias salary by age and by gender
employee_list = []
employee_pay = []
employee_gender_lst = []
employee_age_lst = []
for key,employee in enumerate(employees):
    employee_list.append(employee['name'])
    employee_gender_lst.append(employee['gender'].lower())
    if employee['hourly_rate'] < 0:
        employee['hourly_rate'] = 20
    if employee['gender'].lower() == 'female':
        employee['hourly_rate'] *= random.choice([1.3, 1.6, 1.4, 1.1])
    dob = datetime.strptime(employee['dob'],DATEFORMAT)
    employee['dob'] = datetime(dob.year,dob.month,dob.day,0,0,0)
    age = int(round((datetime.today() - dob).days / 365,0))
    employee_age_lst.append(age)
    employee['hourly_rate'] += round((datetime.today() - employee['dob']).days/900,2)
    employee_pay.append(round(employee['hourly_rate'],2))
print(employees[0:2])
employee_pay_dct = dict(zip(employee_list,employee_pay))
employee_gender_dct = dict(zip(employee_list,employee_gender_lst))
employee_age_dct = dict(zip(employee_list,employee_age_lst))
print(employee_pay)


[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': '1993-06-09', 'hourly_rate': 14}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': '1996-09-07', 'hourly_rate': 20}]
[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': datetime.datetime(1993, 6, 9, 0, 0), 'hourly_rate': 30.24}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': datetime.datetime(1996, 9, 7, 0, 0), 'hourly_rate': 35.32}]
[30.24, 35.32, 26.81, 45.68, 37.69, 42.85, 39.58, 31.17, 30.65, 40.06, 48.18, 35.91, 30.54, 38.74, 29.83, 43.25, 37.59, 41.34, 32.62, 46.62, 42.42, 39.07, 56.58, 27.36, 44.91, 47.58, 41.19, 37.02, 46.43, 27.54, 38.97, 29.85, 24.85, 18.91, 53.53, 50.4, 39.19, 36.75, 43.8, 42.34, 27.11, 34.3, 37.34, 35.25, 41.56,

In [10]:
import numpy as np
from scipy.stats import gamma
from scipy.linalg import cholesky
import random

DATE_FORMAT = "%Y-%m-%s %H:%M:%S"
DATE_FORMAT_SHORT = "%Y-%m-%s"
def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))
    
def determine_visitor(dct):
    name = random.choice(employee_list)
    dct['name'] = name
    dct['gender'] = employee_gender_dct[name].lower()
    dct['age'] = employee_age_dct[name]
    dct['status'] = random.choice(['guest','guest','member'])
    dct['registration_no'] = 'guest'
    if dct['status'] == 'member':
        dct['registration_no'] = randomString()
    
    return dct

def determine_sales_behaviour(dct,area):
    name = random.choice(employee_list)
    
    dct[area+'_employee_name'] = name
    dct[area+'_employee_gender'] = employee_gender_dct[name].lower()
    dct[area+'_employee_age'] = employee_age_dct[name]
    scalar = 1
    if area == 'bar':
        if dct['gender'] == 'female':
            scalar *= 4
        scalar += (dct['age'] - 18) / 4
        items_len = int(gamma.rvs(a=scalar, size=1))
        if items_len < 0:
            items_len = random.randint(1,4)
    else:
        scalar = 5
        if dct['gender'] == 'female':
            scalar *= 2            
        items_len = int(gamma.rvs(a=scalar, size=1))
        if items_len < 0:
            items_len = random.randint(1,4)
    return items_len, dct
        
    

dct2 = {}
dct2['food'] = {
    'salad':
    {
        'caesar':22.00,
        'green':25
    },
    'sandwich':{
        'cutter':14.00,
        'hot dog':8.00
    },
    'finger food':{
        'meatballs':14.00,
        'wings':12.00
    }
}
dct2['drink'] = {
    'vodka':{
        'sky':16.00,
        'grey goose':18.00
    },
    'scotch':{'JW Black':18.00,'glenfiddich':17.00},

    'beer':{'carib':7.00,'stag':8.00,'stag':8.00,'deputy':4.00,'deputy':4.00,'deputy':4.00,'deputy':4.00}
}

dct3 = {
    'paddle board' : 20.00,
    'chair' : 10.00,
    'umbrella' : 13.00,
    'kayak': 20.00
    
}

import random
import string

# or a function
def gen_datetime(min_year=2016, max_year=2020):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random.random()

collection_composite = db['bcc_composite']
def rental_behaviour(dct1):
    item_dct = {}
    composite_lst = []
    timestamp = gen_datetime()
    dct1['visit_start'] = timestamp

    item_dct['tab_opened_at'] = dct1['visit_start'] + timedelta(minutes=3)
    item_dct['visit_tab_id_rental'] = datetime.strftime(item_dct['tab_opened_at'],DATE_FORMAT_SHORT) +'_'+dct1['registration_no']
    visit_len = int(gamma.rvs(a=120, size=1))
    if visit_len < 0:
        visit_len = 64
    dct1['visit_end'] = dct1['visit_start'] + timedelta(minutes=visit_len)
    dct1['visit_duration'] = dct1['visit_end'] - dct1['visit_start']
    dct1['visit_duration'] = round(dct1['visit_duration'].total_seconds()/60,0)
    # rentals
    items_len,employee_dct = determine_sales_behaviour(dct1,'rental')
    #print('rentals:',items_len)
    composite_lst = []
    for item in range(0,items_len+1):
        for key,value in dct1.items():
            item_dct[key] = value
        item_dct['_id'] = ''.join(random.choices(string.ascii_letters + string.digits, k=20))
        item_dct['area'] = 'beach rentals'
        
        item_dct['item'] = random.choice(['paddle board','kayak','kayak','chair','chair',
                                      'chair','chair','chair','chair','chair','umbrella','umbrella','umbrella'])
        if item_dct['item'] in ['paddle board','kayak']:
            
            item_dct['category'] = 'watersports'
        else:
            item_dct['category'] = 'relaxation'
        item_dct['rental_tab'] = dct3[item_dct['item']]
        collection_composite.insert_one(item_dct)
    return item_dct

def bar_behaviour(dct1):

     # rentals
    items_len,employee_dct = determine_sales_behaviour(dct1,'bar')
    composite_lst = []
    for item in range(0,items_len+1):
        item_dct = {}
        item_dct['_id'] = ''.join(random.choices(string.ascii_letters + string.digits, k=20))
        item_dct['tab_opened_at'] = dct1['visit_start'] + timedelta(minutes=40)
        
       
        item_dct['area'] = 'bar'
        item_dct['category'] = random.choice(['food','drink','drink','drink'])
        item_dct['bar_tab'] = 0
       
        item_type = random.choice(list(dct2[item_dct['category']].keys()))
        item_dct['comestible_type'] = item_type
        lst = list(dct2[item_dct['category']][item_type].keys())
        item_name = random.choice(lst)
        item_dct['item'] = item_name
        item_dct['price'] = dct2[item_dct['category']][item_type][item_name] 
        item_dct['amount'] = random.choice([1,1,1,1,1,2,2,3])
        item_dct['bar_tab'] += item_dct['price'] * item_dct['amount']
        
        item_dct['tab_status'] = random.choice(['closed','closed','closed','open'])
        if item_dct['tab_status'] == 'closed':
            item_dct['payment'] = item_dct['bar_tab']
            item_dct['payment_timestamp'] = dct1['visit_end'] - timedelta(minutes=5)

        #composite_lst.append(item_dct)
     
        collection_composite.insert_one(item_dct)
    

counter = 0
while counter < 12000:
    dct = {}
    dct = determine_visitor(dct)
    dct = rental_behaviour(dct)
    bar_behaviour(dct)
    counter += 1
    #print(counter)

            
        
    